# junk_checker

> This module checks the domain_age of an email address

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| default_exp junk_checker

In [ ]:
#| export
from email_filter.exchange import EmailClient, extract_domain
import json
from datetime import datetime
import os

In [ ]:
datetime(2023, 9, 21).strftime('%y/%m/%d %H:%M:%S')

'23/09/21 00:00:00'

In [ ]:
datetime.strptime('23/09/21 00:00:00','%y/%m/%d %H:%M:%S')

datetime.datetime(2023, 9, 21, 0, 0)

In [ ]:
#| export

import os
import json

class DataStorage:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
    
    def _get_file_path(self, file_name):
        return os.path.join(self.folder_path, file_name)
    
    def write_to_json(self, file_name, data):
        file_path = self._get_file_path(file_name)
        with open(file_path, 'w') as f:
            json.dump(data, f, indent = 2)
    
    def read_from_json(self, file_name):
        file_path = self._get_file_path(file_name)
        if not os.path.exists(file_path):
            return None
        with open(file_path, 'r') as f:
            return json.load(f)
    




In [ ]:
#| export
class JunkChecker:
    def __init__(self, email_client, storage):
        self.client = email_client

    def get_last_update_date(self):
        last_update = storage.read_from_json("last_update.json")
        if not last_update:
            last_update = datetime(2020, 7, 1)
        return last_update

    def update_whitelists(self):
        last_update = self.get_last_update_date()
        emails = email_client.get_emails(from_folder = "Sent Items", since_date=last_update, fetch_body = False)
        addresses = [email.recipient + email.cc + email.bcc for email in emails]
        addresses = list(set([a.lower() for alist in addresses for a in alist]))
        domains = list(set([extract_domain(address) for address in addresses]))
        storage.write_to_json("address_whitelist.json", addresses)
        storage.write_to_json("domain_whitelist.json", domains)
        
        

In [ ]:
#| notest

USERNAME = os.environ.get("EXCHANGE_USER")
PASSWORD = os.environ.get("EXCHANGE_PASSWORD")

# Usage example
email_client = EmailClient("msx.tu-dresden.de", 993, USERNAME, PASSWORD)
storage = DataStorage("../data")
junk_checker = JunkChecker(email_client, storage)
junk_checker.update_whitelists()
